# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Составим первое впечатление о данных. Импортируем библиотеку `pandas `, считаем датасет и сохраним его в переменную `data `.
Выведем на экран первые 20 строк таблицы. Этого должно быть достаточно для первого ознакомления и поверхностного знакомства со значениями в колонках.

In [1]:
import pandas as pd
data = pd.read_csv('***')
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем на экран общую информацию о таблице с данными.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов. Типы данных в столбцах: `object`, `float64`, `int64`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Названия колонок указаны корректно, но в самих данных видны огрехи:
* В столбцах `days_employed` и `total_income`данные лучше привести к целочисленному типу, также в столбце `days_employed` видны отрицательные значения а так же выбросы- значения, сильно отличающиеся от остальных. 
* В столбце `education` видны значения, введённые с использованием разных типов регистра. 

**Вывод**

В каждой строке таблицы — данные о заёмщике.  
 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а также некорректные форматы и выбросы. 

Необходимо более детально изучить данные и избавиться от проблем, прежде чем приступить к анализу.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем пропуски.

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски указаны только в столбцах `days_employed` и `total_income` и их одинаковое количество. Можно предположить, что эти значения отсутствуют, т.к. эти люди не трудоустроены и поэтому не имеют дохода. Для начала проверим, совпадают ли эти пропуски или они находятся в разных строках. Заменим пропуски на `0`, затем с помощью двойной логической индексации выведем на экран строки, в которых в обоих столбцах стоит `0` и сравним с количеством пропусков из предыдущей ячейки.

In [5]:
data = data.fillna(0) 
data[(data['days_employed'] == 0) & (data['total_income'] == 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,0.0,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,0.0,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,0.0,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,0.0,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,0.0,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,0.0,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,0.0,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство жилой недвижимости


В отфильтрованной таблице количество строк совпадает с количеством пропущенных значений в изначальной. Но если присмотрется к столбцу `income_type`, то предположение о том, что эти люди не работают отпадает - в таблице есть трудоустроенные. Скорее всего значения в данном случае пропущены из-за технической ошибки и в реальной рабочей задаче следовало бы сообщить об этом разработчикам, делавшим выгрузку.

В случае учебного проекта такой возможности нет и необходимо принять решение, что делать с пропущенными значениями. Так как доля пропусков значительная - примерно 10% от выборки, удалять их нельзя, т.к. это исказит дальнейшие вычисления, лучше заполнить пропуски средними значениями по группам в выборке:
`total_income` будем заполнять средним значением дохода по типу занятости (`income_type`), а `days_employed` - по возрасту (`dob_years`)

Для начала удостоверимся что данные в столбцах, по которым мы будем брать средние значения корректны: для этого посмотрим на них c помошью функции  `value_counts` и `unique`:

In [6]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Типы занятости указаны без повторов, можно считать среднее по каждому.

In [7]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце с возрастом есть 101 нулевое значение. Об общего числа выборки это меньше половины процента, поэтому можно удалить эти данные. Перезапишем таблицу и проверим данные в столбце:

In [8]:
data = data[data['dob_years'] != 0]
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [9]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Код отработал корректно, "ноль" ушёл. Двигаемся к оставшимся двум столбцам. Из ознакомления с данными помним, что в столбце `days_employed` есть выбросы и отрицательные значения. Перед тем, как брать среднее по столбцу, приведём его в порядок.

In [10]:
#Возьмём минимальные и максимальные значения по столбцу.
print(data['days_employed'].min())
print(data['days_employed'].max())

-18388.949900568383
401755.40047533


Если переводить дни в года, то здравый смысл подсказывает, что результаты, превышающие 20000(или 55 календарных лет) дней являются неправдоподобными. Отрицательные значения не превышают этого порога, а положительные превышают до 2-х раз.

Можно предположить, что аномально большие данные указаны в часах. Тогда остаётся поделить большие значения на 24, а отрицательные-заменить значениями по модулю.

In [11]:
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed']>20000,'days_employed'] = data.loc[data['days_employed']>20000,'days_employed']/24
#проверим что получилось:
#помним, что нужно учитывать значения больше 0, так мы заменили на него пропуски:
print(data.loc[data['days_employed']>0,'days_employed'] .min())
print(data['days_employed'].max())

24.14163324048118
18388.949900568383


Теперь значения похожи на адекватные. Код отработал верно, данные можно использовать для данной задачи(помним, что их также следует перевести в формат целочисленных значений)

Гляда на количество значений по возрасту мы видим, что оно не равномерное - лучше собрать возрастные группы, например начиная с 30 лет и с шагом по 15 лет, посчитать по ним среднее и проставить циклом в столбец `days_employed`. 

In [12]:
# создаём группы из значений больше нуля:
# до 30 лет включительно
thirty = data[data['dob_years']<=30]
thirty = thirty[thirty['days_employed']!=0]

# до 45 лет включительно
forty_five = data[data['dob_years']<=45]
forty_five = forty_five[forty_five['dob_years']>=31]
forty_five = forty_five[forty_five['days_employed']!=0]

# до 60 лет включительно
sixty = data[data['dob_years']<=60]
sixty = sixty[sixty['dob_years']>=46]
sixty = sixty[sixty['days_employed']!=0]

# старше 60
sixty_plus = data[data['dob_years']>60]
sixty_plus = sixty_plus[sixty_plus['days_employed']!=0]

# находим средний стаж для каждой группы:
thirty_employed_mean = thirty['days_employed'].mean()
forty_five_employed_mean = forty_five['days_employed'].mean()
sixty_employed_mean = sixty['days_employed'].mean()
sixty_plus_employed_mean = sixty_plus['days_employed'].mean()

#проверим, что получилось
print(thirty_employed_mean, forty_five_employed_mean, sixty_employed_mean, sixty_plus_employed_mean, sep='\n')

1308.2771216454587
2413.130571379816
6622.754545281866
12879.092659135336


Среднее значение старшей возрастной группы неожиданно больше предыдущей. Но я проверил корректность кода и данные отражают реальное распределение по выборке. Заполним нули в стоблце со стажем средними значениями:

In [13]:
data.loc[(data['days_employed'] == 0) & (data['dob_years']<=30), 'days_employed'] = thirty_employed_mean
data.loc[(data['days_employed'] == 0) & (data['dob_years']<=45), 'days_employed'] = forty_five_employed_mean
data.loc[(data['days_employed'] == 0) & (data['dob_years']<=60), 'days_employed'] = sixty_employed_mean
data.loc[(data['days_employed'] == 0) & (data['dob_years']>60), 'days_employed'] = sixty_plus_employed_mean


Проверим, как отработал код вызовем строки с нулевым значением и заодно проверим минимальное значение по столбцу:

In [14]:
print(data['days_employed'].min())
data[data['days_employed'] == 0].count()

24.14163324048118


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Минимальное значение больше нуля, а табличка с нулями пустая, похоже мы справились, можно двигаться дальше

Проверим столбец с доходами:

In [15]:
display(data.loc[data['total_income']>0,'total_income'] .min())
data['total_income'].max()

20667.26379327158

2265604.028722744

Явных выбросов не видно. Можно считать средний доход по типу занятости и подставлять в пропущенные значения с заработной платой. Проверим количество строк с нулями до и после:

In [16]:
data[data['total_income']==0].count()

children            2164
days_employed       2164
dob_years           2164
education           2164
education_id        2164
family_status       2164
family_status_id    2164
gender              2164
income_type         2164
debt                2164
total_income        2164
purpose             2164
dtype: int64

In [17]:
#Создаём список с названиями профессий:
income_type = ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']

#Создаём пустой список для занесения среднего:
income_type_mean = []

#Пишем цикл для подсчёта среднего:
for i_type in income_type:
    df = data[data['income_type'] == i_type]
    income_type_mean.append(df['total_income'].mean())

#Проставляем средний доход по типу занятости в значения с нулями:
#В списке income_type_mean индекс соответсвует типу занятости в списке income_type
for i in range(len(income_type)):
    data.loc[(data['total_income'] == 0) & (data['income_type']==income_type[i]), 'total_income'] = income_type_mean[i]

In [18]:
data[data['total_income']==0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Код отработал корректно. Теперь посмотрим на оставшиеся столбцы на предмет неявных пропусков:

In [19]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видны некорректные значения `-1` и `20`. 
Можно предположить что корректные значения это `1` и `2`, а минус и лишний ноль появились случайно при вводе с клавиатуры.

Заменим данные на предполагаемо корректные(в реальной задаче также перед/вместо этого действия необходимо было бы обратиться к разработчикам). 

Для начала посчитаем количество этих значений:

In [20]:
print(data.loc[data['children']==20,'children'].count())
data.loc[data['children']==-1,'children'].count()

75


47

In [21]:
#Заменим некорректные значения на предполагаемо верные:
data.loc[data['children']==20,'children'] = 2
data.loc[data['children']==-1,'children'] = 1

Снова считаем количество значений для проверки: 

In [22]:
print(data.loc[data['children']==20,'children'].count())
data.loc[data['children']==-1,'children'].count()

0


0

Код сработал. Следующий столбец с информацией об образовании `education`.

In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Столбец содержит неявные дубликаты из-за регистра. В дальнейшем их также необходимо будет обработать. Двигаемся дальше - `family_status_id`. 

In [24]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

Значения выглядят корректно и их количество совпадает к количеством уникальных значений(корректных) в столбце `education`. Следующий столбец- `family_status`.

In [25]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Значение `Не женат / не замужем`лучше привести к нижнему регистру, как и остальные значения по столбцу. Пропусков и некорректных значений нет, идём дальше - столбец `family_status_id`.

In [26]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Некорректных значений нет, количество совпадает с количеством в столбце о семейном положении. Можно идти дальше - `gender`.

In [27]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Обнаружено значение `XNA`, посмотрим количество таких значений:

In [28]:
data.loc[data['gender']== 'XNA','gender'].count()

1

В таблице с таким значением всего одна строка. Можно её убрать. Перезапишем таблицу и проверим результат:

In [29]:
data = data.loc[data['gender']!= 'XNA']

In [30]:
data['gender'].unique()

array(['F', 'M'], dtype=object)

В столбце остались только корректные значения, можно двигаться дальше - к столбцу `debt`.

In [31]:
data['debt'].unique()

array([0, 1])

Как и предпологалось, в столбце только два значения. Перейдём к последнему столбцу - цель кредита `purpose`.

In [32]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Значения следует лемматизировать перед анализом, но скрытых пропусков нет. 

**Вывод**

* Обнаружены пропуски в двух столбцах(`days_employed` и `total_income`), которые были заменены средним значением по группам. 
* Также обнаружены и удалены нули в столбце с возрастом и некорректное значение в столбце с полом.
* В столбцах `days_employed` и `children` аномальные и некорректные значения были заменены условно корректными.

Работу с пропусками можно считать завершённой, пора переходить к следующим шагам по предобработке данных.

### Замена типа данных

Значения в столбцах `days_employed` и `total_income` лучше привести к целочисленному типу (конечно, зарплату могут платить с точностью до копеек, но такими значениями можно принебречь).
Используем метод `astype()`, так как им удобно менять тип данных сразу на целочисленный(проходил я пока только его и numeric(), а он преобразует в вещестненный тип, который для данной задачи не подходит).

Изменим типы данных с `float` на `int` и посмотрим на типы данных до и после изменения.

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null float64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null float64
purpose             21423 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [34]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**
Тип данных изменён успешно, переходим к поиску и обработке дубликатов.

### Обработка дубликатов

Прежде чем искать явные дубликаты вспомним, что в данных с информацией об образовании и семейном положении есть значения с разным типом регистра. Приведём их к единому формату записи и уже затем найдём и удалим дубликаты, иначе эту процедуру прийдется повторять два раза, а дубликаты нам не к чему)

Взглянем ещё раз на уникальные значения столбца с информацией об образовании:

In [36]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведём значения к единому регистру вызовом метода str.lower() и перезапишем таблицу:

In [37]:
data.loc[:,'education'] = data.loc[:,'education'].str.lower()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Проверим, что получилось:

In [38]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь количество значений соответствует количеству в столбце `education_id`, можно переходить к столбцу с семейным положением:

In [39]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Изменение регистра не приведёт к результату предыдущего столбца, но может упростить работу со значениями в дальнейшем, поэтому всё-же проведём такую же процедуру:

In [40]:
data.loc[:,'family_status'] = data.loc[:,'family_status'].str.lower()

In [41]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Найдём количество явных дубликатов с помощью совместного использования методов `duplicated()` и `sum()`.

Затем удалим дубликаты с помощью метода `drop_duplicates()` и проверим, что дубликатов больше не осталось.

In [42]:
data.duplicated().sum()

71

Удалим дубликаты и перезапишем таблицу:

In [43]:
data = data.drop_duplicates().reset_index(drop=True) 

Проверим, что дубликатов не осталось:

In [44]:
data.duplicated().sum()

0

Код отработал корректно, задача по удалению явных дубликатов выполнена. 

**Вывод**

Дубликаты могли появится в датасете как из-за технической проблемы, так и из-за человеческого фактора.
Также в данных отсутствует столбец с уникальным номером пользователя, поэтому также нельзя уверенно говорить о том, что один и тот же человек не внесён в базу два и более раз. Теоретически, цели у разных кредитов, очевидно, будут разные, рабочий стаж на момент обращения так же изменится, и вполне допустимо что может изменится и доход занимателя. Соответсвенно, при определённом количестве таких 'одинаковых' заёмщиков, выводы о ключевых гипотезах могут быть также искажены. Поэтому в реальных задачах лучше также проработать этот вопрос.

В этом проекте я следую изначально заданной структуре и логике работы с датасетом. Но мне видится более логичным удалить дубликаты до этапа заполнения пропусков в столбцах `days_employed` и `total_income`, так как повторяющиеся данные могут исказить реальное среднее/медиану по столбцам.

### Лемматизация

Ещё раз выведем на экран список уникальных целей кредита, отсортированный в алфавитном порядке:

In [45]:
data['purpose'].sort_values().unique() 

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Проведём лемматизацию значений. Для этого нужно в начале переведём список со значениями в строку с помощью метода `join()`, разделим слова пробелом и проверим результат:

In [46]:
purposes = ' '.join(data['purpose'].unique())
purposes

'покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием'

Лемматизируем получившуюся строку и посчитаем количество упоминаний каждого из обработанных слов. 

Для этого импортируем функцию `Mystem` из библиотеки `pymystem3`

Затем для подсчёта лемматизированных строк вызовем контейнер Counter из модуля `collections`.

In [47]:
#Ипрортируем Mystem и сохраняем его в переменной 'm' для удобства
from pymystem3 import Mystem
m = Mystem() 

# Лемматизируем строку с целями и сохраняем в переменной 'lemmas'
lemmas = m.lemmatize(purposes)
from collections import Counter

#выводим значения с количеством упоминаний на экран:
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Из часто встречающихся лемматизированных целей можно выделить 4 основные категории: `недвижимость`(сюда я отнёс слова 'недвижимость' и 'жилье'), `автомобиль`, `образование`, `свадьба`.
    

Напишем функцию для метода `apply()`, которая присвоит каждому кредиту категорию, исходя из его цели.

In [48]:
def purpose_key(row):
    
    #Лемматизирем строку
    lemmas = m.lemmatize(row)
    
    #Сравниваем получившийся список с каждой из выделенных категорий  и возвращаем совпавшее значение
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'цель не определена'

Создадим дополнительные столбец `purpose_key`и заполним его получившимися категориями.

Применим созданную только что функцию к столбцу с целями кредита с помощью `apply()`

Перезапишем таблицу и выведем результат на экран

In [50]:
data['purpose_key'] = data['purpose'].apply(purpose_key)
data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_key
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21348,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21349,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21350,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


Функция сработала корректно. Лемматизация окончена.

**Вывод**

Создан новый столбец с указанием категории цели кредита, что значительно упростит работу с этим параметром.

### Категоризация данных

После анализа данных мы должны дать ответы на 4 вопроса:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
Для ответа на этот вопрос достаточно создать таблицу с количеством детей и наличием задолженности.

* Есть ли зависимость между семейным положением и возвратом кредита в срок?
Здесь удобнее будет использовать столбцы `debt` и `family_status_id` для удобной работы с данными, а также отдельно создать 'словарь' со столбцами `family_status_id` и `family_status`, для ориентации в значениях.

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Для ответа на этот вопрос будет сложно и неудобно анализировать каждое значение дохода. Лучше выделить группы, присвоить группу каждой строке, выделить их в отдельный столбец и уже после этого искать ответы на вопросы исследования.

* Как разные цели кредита влияют на его возврат в срок?
Для последнего вопроса можно создать таблицу с 2-мя столбцами: информация о задолженности и ключевому слову для целей кредита.

Перед ответами на вопросы таблицу лучше представить в более компактном виде. Это упростит визуальную работу с таблицей, уменьшит размер файла и время обработки данных. Создадим 'словари' - таблицы с указанием развёрнутого значения параметра и его номера идентификатора, либо ключевого слова. 

Для ответов на вопросы исследования нам понадобятся:

'Словарь' с идентификатором семейного положения. Создадим его из столбцов `family_status_id` и `family_status`. Ешё раз посмотрим а его значения с помощью метода `drop_duplicates()`

In [51]:
family_status = data[['family_status', 'family_status_id']]
family_status =family_status.drop_duplicates().reset_index(drop=True) 
family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Значения записаны корректно.

Следующий 'словарь' - таблица со столбцами `purpose` и `purpose_key`. Создадим её и отсортируем по ключевым словам

In [52]:
data[['purpose','purpose_key']].drop_duplicates().sort_values(by = 'purpose_key').reset_index(drop=True)

,purpose,purpose_key
0,приобретение автомобиля,автомобиль
1,на покупку автомобиля,автомобиль
2,сделка с автомобилем,автомобиль
3,свой автомобиль,автомобиль
4,автомобиль,автомобиль
5,сделка с подержанным автомобилем,автомобиль
6,автомобили,автомобиль
7,на покупку подержанного автомобиля,автомобиль
8,на покупку своего автомобиля,автомобиль
9,покупка жилья,недвижимость


'Словари' готовы. 

Перед началом анализа создадим ещё один столбец - с указанием категорий дохода.

Посмотрим на ключевые показатели столбца с доходом: минимальное и максимальное значение, среднее и медиана.

In [53]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].median())
data['total_income'].mean()

20667
2265604
145357.0


165809.19497002623

Из-за высоких значений среднее по столбцу искажено в большую сторону и, если бы требовалось разделить на группы именно по данным столбца, лучше бы подошла медиана. Но я считаю что корректней использовать градацию дохода по региону(стране), откуда взяты данные. В данном проекте я позволю себе ориентироваться на субьективные значения, т.к. проект учебный.

Разделим уровни дохода следующим образом:
* до 80 тысяч рублей - ниже среднего
* от 80001 до 140 тысяч рублей - средний
* выше 140 тысяч рублей - высокий

Напишем функцию `salary_rate`, принимающую в качестве аргумента значение дохода и выдающее его категорию:

In [54]:
def salary_rate(row):
    if row <= 80000:
        return 'ниже среднего'
    if row <= 140000:
        return 'средний'
    return 'высокий'

Проверим работу функции:

In [55]:
print(salary_rate(15))
print(salary_rate(130000))
salary_rate(140001)

ниже среднего
средний


'высокий'

Функция работает корректно, теперь применим её к столбцу с доходами и запишем результаты в столбец `salary_rate`.

Перезапишем таблицу и выведем её на экран.

In [56]:
data['salary_rate'] = data['total_income'].apply(salary_rate)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_key,salary_rate
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,высокий
21348,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,высокий
21349,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,средний
21350,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,высокий


Теперь можно смело переходить к анализу и ответам на вопросы.

**Вывод**

Я создал 'словари' и категоризировал ключевые данные для исследования. Можно переходить к анализу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим сводную таблицу для ответа на первый вопрос - с указанием количества детей и количеством просрочек по кредиту.

Сгруппируем её и посчитаем количетсво значений. Также для нагладности добавим количество кредиторов по количеству детей и средний процент невозврата кредита в срок. Для этого нам понадобится `aggfunc`.

In [57]:
#создаём сводную таблицу, в качестве столбца с индексами используем столбец с количеством детей:
first_question = data.pivot_table(index =  ['children'], values = 'debt', aggfunc = ['sum', 'count','mean'])

#добавим названия столбцов: задолжность, количество кредиторов и процент:
first_question.columns = ['debt', 'total', '%']

#отсортируем таблицу по стобцу со средним процентом в порядке убывания и выведем получившуюся таблицу на экран:
first_question = first_question.sort_values(by = ['%'], ascending = False)
first_question.style.format({'%': '{:.2%}'})

,debt,total,%
children,,,
4,4,41,9.76%
2,202,2114,9.56%
1,442,4839,9.13%
3,27,328,8.23%
0,1058,14021,7.55%
5,0,9,0.00%


Из получившейся таблицы можно сделать любопытные выводы:

* Люди, имеющие троих детей в целом реже допускают просрочки, чем кредиторы с одним, двумя и четырьмя.

* Кредиторы без детей в среднем чаще гасят кредит в срок.

Получившаяся таблица косвенно отвечает на изначальный вопрос но точный ответ должен содержать средний процент по всем заёмщикам имеющим детей и кредиторам без детей. Второе значение нам уже известно. Осталось только найти среднее значение по столбцу с количеством кредиторов с детьми при помощи логической индексации и функции `mean()`:

In [59]:
have_children = data.loc[data['children']!=0, 'debt'].mean()
without_children = data.loc[data['children']==0, 'debt'].mean()
print(without_children)
have_children

0.07545824120961415


0.09207475105715456

Для наглядности создадим маленькую таблицу и занесём туда значения:

In [69]:
data['children_group'] = data['children'].apply(lambda x: 'have_children' if x > 0 else 'w/o_children')
data.groupby('children_group')[['debt']].mean().style.format({'debt': '{:.2%}'})

,debt
children_group,
have_children,9.21%
w/o_children,7.55%


**Вывод**

* В среднем люди без детей примерно на 1,7 процента реже допускают просрочки по кредитам.
* Количетсво детей незначительно влияет на показатель погашения кредита вовремя.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу по примеру прошлой, для удобства добавим в неё текстовое название семейного положения с помощью функции `merge()`:

In [70]:
#создаём сводную таблицу, в качестве столбца с индексами используем столбец с уникальным номером семейного положения:
second_question = data.pivot_table(index =  ['family_status_id'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])

#добавим названия столбцов: задолжность, количество кредиторов и процент:
second_question.columns = ['debt', 'total', '%']

#присоединим текстовые обозначения семейного положения:
second_question = second_question.merge(family_status, on = 'family_status_id', how='left')

#отсортируем таблицу по стобцу со средним процентом в порядке убывания и выведем получившуюся таблицу на экран:
second_question = second_question.sort_values(by = ['%'], ascending = False)
second_question.style.format({'%': '{:.2%}'})



,family_status_id,debt,total,%,family_status
4,4,273,2794,9.77%,не женат / не замужем
1,1,386,4129,9.35%,гражданский брак
0,0,927,12290,7.54%,женат / замужем
3,3,85,1185,7.17%,в разводе
2,2,62,954,6.50%,вдовец / вдова


**Вывод**

* Самая большая частота просрочек кредита у неженатых людей - почти каждый сотый кредитор имеет просрочки*
* Затем очень близко к этой категории находятся люди, живущие в гражданском браке, затем люди в браке, затем разведённые.
* Как ни странно, самые порядочные кредиторы - вдовы и вдовцы. Разница с неженатыми - примерно 3,3 процента.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Повторим знакомую процедуру:

In [71]:
#создаём сводную таблицу, в качестве столбца с индексами используем столбец с категорией дохода:
third_question = data.pivot_table(index =  ['salary_rate'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])

#добавим названия столбцов: задолжность, количество кредиторов и процент:
third_question.columns = ['debt', 'total', '%']

#отсортируем таблицу по стобцу со средним процентом в порядке убывания и выведем получившуюся таблицу на экран:
third_question = third_question.sort_values(by = ['%'], ascending = False)
third_question.style.format({'%': '{:.2%}'})


,debt,total,%
salary_rate,,,
средний,603,7179,8.40%
высокий,956,11909,8.03%
ниже среднего,174,2264,7.69%


**Вывод**

Результаты снова получились неочевидными:
* люди с самым низким доходом оказались самыми порядочными кредиторами.
* люди со средним доходом чаще других допускают просрочки. Значение людей с высоким доходом довольно близко к ним.

- Как разные цели кредита влияют на его возврат в срок?

И снова нам поможет сводная таблица:

In [72]:
#создаём сводную таблицу, в качестве столбца с индексами используем столбец с ключевым словом цели кредита:
fourth_question = data.pivot_table(index =  ['purpose_key'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])

#добавим названия столбцов: задолжность, количество кредиторов и процент:
fourth_question.columns = ['debt', 'total', '%']

#отсортируем таблицу по стобцу со средним процентом в порядке убывания и выведем получившуюся таблицу на экран:
fourth_question = fourth_question.sort_values(by = ['%'], ascending = False)
fourth_question.style.format({'%': '{:.2%}'})

,debt,total,%
purpose_key,,,
автомобиль,400,4284,9.34%
образование,370,3995,9.26%
свадьба,184,2310,7.97%
недвижимость,779,10763,7.24%


**Вывод**

* Люди, обращающиеся за займом на автомобиль и образование - самые ненадёжные кредиторы.
* Кредиторы с целями: свадьба и недвижимость допускают просрочки реже. 
* Разница между целями 'автомобиль' и 'недвижимость' - 2,1 процента.

## Шаг 4. Общий вывод

Во время предобработки данных были обнаружены пропуски в двух столбцах(days_employed и total_income), которые были заменены средним значением по группам. К замене данных пришлось прибегнуть, т.к. количество пропусков составляло порядка 10% и удаление могла исказить результы анализа. Данные пропуски похожи на системную ошибку.

Также обнаружены и нули в столбце с возрастом и некорректное значение в столбце с полом. В общем количество этих данных составляло около процента от выборке, поэтому я принял решение удалить эти значения. Некорректное значение в столбце с полом было вызвано системной ошибкой, в столбце с возрастом - больше похоже на человеческий фактор. Реккомендую также обратиться к команде, ответственной за выгрузку с просьбой о проверке процедуры заполнения значений.

В столбцах `days_employed` и `children` аномальные и некорректные значения были заменены условно корректными. 

Реккомендация после проверки данных - обратиться к команде, ответственной за выгрузку с просьбой о проверке процедуры заполнения значений на предмет устранения/сведения к минимуму ошибок, связанных с человеческим фактором и проверки системы и алгоритмов процедения выгрузки в данных, т.к. в них явно наблюдаются ошибки системного характера.

Анализ имеющихся данных дал ответы на 4 вопроса:

1. Люди без детей чаще возвращают кредит в срок. 

2. Вдовцы и вдовы возвращают кредит чаще, чем люди с другим семейным положением. Больше всего просрочек у неженатых и людей в гражданском браке. Женатые и в разводе в середине 'рейтинга'

3. Люди со средним и высоким доходом чаще имеют задолженность, чем люди с низким.
 
4. Кредиторы, занимающие на свадьбу и недвижимость чаще возвращают деньги в срок, чем кредиторы с целями 'автомобиль' и 'образование'.

Исследование можно считать завершённым.